In [1]:
import numpy as np

In [2]:
def sigmoid(x):
    return np.exp(x) / (np.exp(x) + 1)

In [3]:
class Logistic2nd:
    
    def __init__(self, rate=0.001, cycles=1):
        self.rate = rate
        self.cycles = cycles
    
    def run(self, points, ids):
        self.theta = np.zeros(np.size(points[0]))
        self.bias = 0
        
        self.best_theta = np.zeros(np.size(points[0]))
        self.best_bias = 0
        self.best_miss_count = len(ids)
        self.best_best_miss = []
        
        self.loss = np.zeros(self.cycles)
        
        for i in range(self.cycles):
            for point, truth in zip(points, ids):
                error = truth - self.predict(point)
                self.compute_loss(i, point, truth)
                
                shift = abs(error) * self.rate
                dot = np.dot(point, self.theta) + self.bias
                
                pointb = (np.append([1], point)).reshape(len(point)+1,1)
                thetab = (np.append([self.bias],self.theta)).reshape(len(self.theta)+1,1)
                
                g = pointb * (sigmoid(dot) - truth)
                H = np.dot(pointb, np.transpose(pointb)) * sigmoid(-1 * dot) * sigmoid(dot)
                print(H)
                Hi = np.linalg.inv(H)
                Hig = np.dot(Hi,g)
                
                self.bias -= shift * Hig[0]
                self.theta -= shift * Hig[1:]
            self.compute_miss(points, ids)
        
        self.report(points, ids)
        return self
    
    def compute_loss(self, i, x, y):
        dot = np.dot(x, self.theta)
        self.loss[i] -= y * np.log(sigmoid(dot)) + (1 - y) * np.log(sigmoid(-1 * dot))
    
    def predict(self, ds):
        return np.where(sigmoid(np.dot(ds, self.theta) + self.bias) > 0.5, 1, 0)
    
    def compute_miss(self, points, ids):
        miss_count = 0
        miss = []
        for point, truth in zip(points, ids):
            prediction = self.predict(point)
            if truth - prediction != 0:
                miss_count += 1
                miss.append(point)
        if miss_count <= self.best_miss_count:
            self.best_miss_count = miss_count
            self.best_miss = miss
            self.best_bias = self.bias
            self.best_theta = self.theta
    
    def report(self, points, ids):
        miss = []
        for point, truth in zip(points, ids):
            prediction = self.predict(point)
            if truth - prediction != 0:
                miss.append([point, prediction, truth])
        if len(miss) > 0:
            print("Mislabeled:")
            for c in miss:
                print("{0}, prediction: {1}, truth: {2}".format(c[0], c[1], c[2]))

In [4]:
log = Logistic2nd()

In [5]:
data = np.load('Data1/data.npz')
x = data['x']
y = data['y']

In [6]:
out = log.run(x,y)

[[0.25       0.01211319 0.27685723]
 [0.01211319 0.00058692 0.0134145 ]
 [0.27685723 0.0134145  0.3065997 ]]


LinAlgError: Singular matrix

In [ ]:
print(out.theta)
print(out.bias)
print(out.best_theta)
print(out.best_bias)
print(out.best_miss)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions

In [ ]:
plt.plot(out.loss)
plt.xscale('log')
plt.xlabel('Cycles')
plt.ylabel('Loss')

In [ ]:
if len(x[0]) == 2:
    plot_decision_regions(x, y, clf=log)